In [10]:
# Importing all the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [11]:
# Reading the data

df= pd.read_csv('C:/Users/roboc/Downloads/trainer.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
df.shape

(159571, 8)

In [13]:
df.tail()
df['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [14]:
df.iloc[5]

id                                                00025465d4725e87
comment_text     "\n\nCongratulations from me as well, use the ...
toxic                                                            0
severe_toxic                                                     0
obscene                                                          0
threat                                                           0
insult                                                           0
identity_hate                                                    0
Name: 5, dtype: object

In [15]:
# Text to vector Conversion

from tensorflow.keras.layers import TextVectorization

In [16]:
unique_words = 200000

In [17]:
# Creating an instance for TextVectorization

vectorizer = TextVectorization(max_tokens=unique_words, output_mode='int',output_sequence_length=1800)

In [18]:
# Convert into X and Y for training and testing spllit

X = df['comment_text'].values
y = df.iloc[:,2:].values
X

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [19]:
# Fitting the model and training the data with text X

vectorizer.adapt(X)

In [20]:
vectorized_text = vectorizer(X)
vectorized_text[0]

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([645,  76,   2, ...,   0,   0,   0], dtype=int64)>

In [21]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [22]:
# Creating a pipeline for the data-flow + Batching the data

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [23]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [24]:
# Printing Batches of data

dataset.as_numpy_iterator().next()

(array([[    8,   208,   365, ...,     0,     0,     0],
        [  729, 30635,  1211, ...,     0,     0,     0],
        [  117,     4,     2, ...,     0,     0,     0],
        ...,
        [ 1593,     7,  1475, ...,     0,     0,     0],
        [  726,    62,  1171, ...,     0,     0,     0],
        [   12,   534,  3141, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [25]:
# Converting the data into 70% training data, 20% training data and 10% validation data

train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [26]:
# Importing Libraries for creating neural network

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from keras.optimizers import Adam

In [27]:
# Creating a neural network with Embedding Layer, Bi-directional LSTM , 3 RELU Layers and Sigmoid Layer

model = Sequential()
model.add(Embedding(unique_words+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [28]:
# Creating 

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
# Seeing whether the modelgot created or not

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [30]:
# Training the model with 1-epoch

history = model.fit(train, epochs=1, validation_data=val)


6981/6981 [==============================] - 6871s 983ms/step - loss: 0.0626 - accuracy: 0.9849 - val_loss: 0.0448 - val_accuracy: 0.9948


In [31]:
history.history

{'loss': [0.06257190555334091],
 'accuracy': [0.9849412441253662],
 'val_loss': [0.04482215642929077],
 'val_accuracy': [0.9948282241821289]}

In [32]:
# Make Predictions

input_text = vectorizer("You are a worthless piece of garbage")


In [33]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [34]:
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 2s 2s/step


array([[0.9896354 , 0.14595059, 0.89141333, 0.01356546, 0.6963134 ,
        0.07543297]], dtype=float32)

In [35]:
# Testing batches

batch_X,batch_Y = test.as_numpy_iterator().next()
batch_Y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [36]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 214ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [37]:
# Evaluate Model

from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [38]:
# Testing the model

for batch in test.as_numpy_iterator():
    x_actual,y_actual = batch
    y_pred = model.predict(x_actual)
    y_actual_flatten = y_actual.flatten()
    y_pred_flatten = y_pred.flatten()
    pre.update_state(y_actual_flatten,y_pred_flatten)
    re.update_state(y_actual_flatten,y_pred_flatten)
    acc.update_state(y_actual_flatten,y_pred_flatten)

1/1 [==============================] - 0s 194ms/step


In [ ]:
# Making a GUI interface for viewing Toxic Comment classification using GRADIO

import gradio as gr

model = tf.keras.models.load_model('toxicity.h5')

def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

# Create the Gradio interface
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                         outputs='text')
# Launch the interface
interface.launch(share=True)
